In [93]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager

In [94]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\natha\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


# Scrape NASA Mars News

In [95]:
# Visit https://redplanetscience.com/
article_url = "https://redplanetscience.com/"
browser.visit(article_url)

time.sleep(1)

In [96]:
# Scape page into Soup
art_html = browser.html
art_soup = bs(art_html, "html.parser")

slide_elem = art_soup.select_one('div.list_text')

In [97]:
slide_elem

<div class="list_text">
<div class="list_date">August 1, 2021</div>
<div class="content_title">How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus </div>
<div class="article_teaser_body">Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.</div>
</div>

In [98]:
# Get latest news title
latest_news_title = slide_elem.find('div', class_='content_title').get_text()
latest_news_title

"How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus "

In [99]:
# Get the paragraph text for latest new title
news_para = slide_elem.find('div', class_='article_teaser_body').get_text()
news_para

'Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.'

# Scape Featured Mars Space Image

In [122]:
# Visit https://spaceimages-mars.com/
image_url = "https://spaceimages-mars.com/"
browser.visit(image_url)

time.sleep(1)

In [123]:
# Scape image_page into Soup
image_html = browser.html
image_soup = bs(image_html, "html.parser")

In [155]:
# Find FULL IMAGE button
button_elem = image_soup.select_one('div.floating_text_area')
button_elem

<div class="floating_text_area">
<h2 class="brand_title">FEATURED IMAGE</h2>
<h1 class="media_feature_title">Dusty Space Cloud</h1>
<br/>
<a class="showimg fancybox-thumbs" href="image/featured/mars2.jpg" target="_blank"> <button class="btn btn-outline-light"> FULL IMAGE</button></a>
</div>

In [153]:
# Get FULL IMAGE link
full_image_button = button_elem.select_one('a').get("href")
full_image_button

'image/featured/mars2.jpg'

In [154]:
# Add FULL IMAGE link to base url to get featured image
featured_image_url = image_url + full_image_button
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

# Scrape Mars Facts

In [157]:
# Get Mars comparison table with pandas 'pd.read_html'

mars_comp_df = pd.read_html('https://galaxyfacts-mars.com/')[0]
mars_comp_df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [158]:
# Rename table columns
mars_comp_df.columns = ['Mars - Earth Comparison', 'Mars', 'Earth']
mars_comp_df.set_index('Mars - Earth Comparison', inplace=True)
mars_comp_df

,Mars,Earth
Mars - Earth Comparison,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [162]:
mars_facts_df = mars_comp_df[1:].reset_index()
mars_facts_df

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-87 to -5 °C,-88 to 58°C


# Scrape Images and Titles of Mars Hemispheres

In [183]:
hem_url = "https://marshemispheres.com/"
browser.visit(hem_url)

time.sleep(1)

In [184]:
hem_html = browser.html
hem_soup = bs(hem_html, "html.parser")

In [185]:
hem_elem = hem_soup.select_one("div.item")
hem_elem

<div class="item">
<a class="itemLink product-item" href="cerberus.html"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>
<div class="description">
<a class="itemLink product-item" href="cerberus.html">
<h3>Cerberus Hemisphere Enhanced</h3>
</a>
<span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/>
<p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p>
</div>
<!-- end description -->
</div>

In [186]:
hem_links = hem_elem.select_one('a').get("href")
hem_links

'cerberus.html'

In [187]:
links_url = hem_url + hem_links
links_url

'https://marshemispheres.com/cerberus.html'

In [190]:
links = browser.find_by_css("a.product-item h3")
links

In [191]:
hemisphere_image_urls = []

links = browser.find_by_css("div.item")

for item in range(len(links)):
    hemisphere = {}
    
    # Loop through each element and click on hemisphere links
    browser.find_by_css("div.item")["href"][0].click()
    
    # Get url for full resolution image of hemisphere
    sample_link = browser.find_link_by_text("Sample").first
    hemisphere["img_url"] = sample_link["href"]
    
    # Get hemisphere title
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Return to previous webpage
    browser.back()

TypeError: 'NoneType' object is not subscriptable

In [168]:
hemisphere_image_urls

[]

In [ ]:
browser.quit()